## Summary 

- DATA USED - Naviga_Entity_Extraction_v002.xlsx


- FILES CREATED - location_norm2raw2, location_raw2norm2, company_norm2raw2, company_raw2norm2, persons_norm2raw2, persons_raw2norm2
               

- Matching algorithms of notebook 'entity_normalization_ver_0_3_6' were combined in a script file Normalization_Helper.py and used, Co Document Ratio was not used. DKE-Intervention/Post-processing was done for all 3 types of entities manually.



In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import json
import math
import copy
import random
from collections import defaultdict, Counter, OrderedDict
from itertools import combinations
import unicodedata
import re
import matplotlib.pyplot as plt
import pickle

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import Levenshtein as lvst
from metaphone import doublemetaphone as dmt_phone
import networkx as nx
import nxviz as nv

In [4]:
from sparse_dot_topn import awesome_cossim_topn as cossim_topn

In [5]:
from Normalisation_Helper import get_df_match,add_match_score,cut_off,get_connected_comp,get_postProcessigUnits
from Normalisation_Helper import choose_normalized,get_comp_final,make_norm_dict,normalize_unicode_to_ascii

In [6]:
df=pd.read_excel("Naviga_Entity_Extraction_v002.xlsx")
df = df.fillna("")

In [7]:
persons= df["Person_Naviga"]
company = df["Company_Naviga"]
location = df["Location_Naviga"]

## Persons


In [22]:
df_persons = get_df_match(persons, 'Person', wiki_match=False, sorting_1st_letters=True)

# of candidate pairs: 2588


In [23]:
weights_person = [4,2,1,1,1]
add_match_score(df_persons, weights=weights_person)

In [24]:
df_persons = cut_off(df_persons)

In [25]:
df_persons.shape

(69, 8)

In [26]:
df_persons.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Beaver Dam,BEAVER DAM,1.0,True,0.997527,1.0,1.0,0.999725
1,Ms. Jones,Mrs. Jones,1.0,True,0.997527,1.0,1.0,0.999725
2,Brian Mahoney,BRIAN MAHONEY,1.0,True,0.997527,1.0,1.0,0.999725
3,Kim Jong-un,Kim Jong Un,1.0,True,0.997527,1.0,1.0,0.999725
4,NADIA BOULANGER,Nadia Boulanger,1.0,True,0.997527,1.0,1.0,0.999725
5,Gov DeWine,Gov. DeWine,1.0,True,0.997527,1.0,1.0,0.999725
6,PLANT SALE,Plant Sale,1.0,True,0.997527,1.0,1.0,0.999725
7,DEAR ABBY,Dear Abby,1.0,True,0.997527,1.0,1.0,0.999725
8,President Carter,Mr. Carter,1.0,True,0.997527,1.0,1.0,0.999725
9,President Trump's,Mrs. Trump's,1.0,True,0.997527,1.0,1.0,0.999725


In [27]:
df_persons.tail(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
59,William Lesh,William Leslie,0.711813,True,8.315280e-07,0.333333,1.000000,0.686732
60,Kenny Porterfield,Kady Porterfield,0.762040,True,1.233946e-04,0.333333,0.791702,0.685924
61,Tod von George Floyd,von George Floyds Tod,0.819859,True,8.315280e-07,0.111111,0.752381,0.682547
62,Mar&#237;a Blanco,Mar&#237;a Brenes,0.712363,True,1.233946e-04,0.250000,1.000000,0.677730
63,Garret Smith,Garrett Smithley,0.723815,True,8.315280e-07,0.250000,0.938889,0.676016
64,Calais Campbell,Chris Campbell,0.711167,True,1.233946e-04,0.333333,0.834643,0.668085
65,Ben Crump,Benjamin Crump,0.735444,True,1.233946e-04,0.200000,0.851852,0.665972
66,Valerie Arkoosh,Valerie A. Arkoosh,0.935304,False,7.310586e-01,0.500000,0.996078,0.663150
67,James Phillips,J. Phillips,0.710630,True,8.315280e-07,0.250000,0.779286,0.652423
68,William Bryan,Bryan Williams,0.776557,True,8.315280e-07,0.083333,0.614850,0.644935


In [28]:
persons_connected = get_connected_comp(df_persons)
connected_comp = persons_connected

In [29]:
connected_comp_short, connected_comp_long, connected_comp_all = get_postProcessigUnits(connected_comp)

In [30]:
print(len(connected_comp_short))
print(len(connected_comp_long))
print(len(connected_comp_all))

63
3
66


In [31]:
connected_comp_short[:20]

[(0, ['Adirondack Wine', 'ADIRONDACK WINE']),
 (1, ['Alma Mahler', 'ALMA MAHLER']),
 (2, ['Antonio Guterres', 'Antnio Guterres']),
 (3, ['BEETHOVEN', 'Beethoven']),
 (4, ['BRENDA BETHUNE', 'Brenda Bethune']),
 (5, ['BRIAN MAHONEY', 'Brian Mahoney']),
 (6, ['Baker MG', 'Mr Baker']),
 (7, ['Beaver Dam', 'BEAVER DAM']),
 (8, ['Beekman Street Art Fair', 'BEEKMAN STREET ART FAIR']),
 (9, ['Ben Crump', 'Benjamin Crump']),
 (10, ['Ben Warpup', 'Brian Warpup']),
 (11, ['Benjamin Harris', 'Benjamin Hardin']),
 (12, ["Bill De Blasio's", "Bill de Blasio's"]),
 (13, ['Breonne Taylor', 'Breonna Taylor']),
 (14, ['Brian Douglass', 'Brian Douglas']),
 (15, ['Bruce Winters', 'Bruce Winten']),
 (16, ['Chris Campbell', 'Calais Campbell']),
 (17, ['Christiano Ronaldo', 'Cristiano Ronaldo']),
 (18, ['Dan Nelson', 'Don Nelson']),
 (19, ['Dave Robertson', 'Dave Roberts'])]

In [32]:
len(connected_comp_long)

3

In [33]:
connected_comp_long

[(0, ['Mr. Jones', 'Mrs. Jones', 'Ms. Jones']),
 (1, ["Mrs. Trump's", "Mr. Trump's", "President Trump's"]),
 (2, ['President Trump', 'Mrs. Trump', 'Mr. Trump'])]

In [34]:
# manual insepction to excude, connect & add
index2exclude = []
index2connect = [(1,2)]
comp2add = []

In [35]:
connected_comp_final = get_comp_final(connected_comp_long, connected_comp_short, is_all=False
                                     , index2connect=index2connect
                                     , index2exclude=index2exclude
                                     , comp2add=comp2add
                                     )

In [36]:
print(len(connected_comp))
print(len(connected_comp_final))

66
65


In [37]:
persons_connected = connected_comp_final

In [38]:
normalized_terms = choose_normalized(persons_connected, choice='long')

In [39]:
raw2normalized, normalized2raw = make_norm_dict(persons_connected, normalized_terms)

In [40]:
print("# of raw (in-snippet) entiteis normalized: {}".format(len(raw2normalized)))
print("# of normalized keys: {}".format(len(normalized2raw)))

# of raw (in-snippet) entiteis normalized: 135
# of normalized keys: 65


In [ ]:
ver_string="2"
f_name = 'persons_matching' + ver_string
df_persons.to_csv("corpus/" + f_name + ".csv")

In [41]:
persons_raw2norm = raw2normalized
persons_norm2raw = normalized2raw

In [42]:
list(persons_raw2norm.items())[:5]

[('Mr. Jones', 'Mrs. Jones'),
 ('Mrs. Jones', 'Mrs. Jones'),
 ('Ms. Jones', 'Mrs. Jones'),
 ("Mrs. Trump's", "President Trump's"),
 ("Mr. Trump's", "President Trump's")]

In [43]:
list(persons_norm2raw.items())[:5]

[('Mrs. Jones', ['Mr. Jones', 'Mrs. Jones', 'Ms. Jones']),
 ("President Trump's",
  ["Mrs. Trump's",
   "Mr. Trump's",
   "President Trump's",
   'President Trump',
   'Mrs. Trump',
   'Mr. Trump']),
 ('ADIRONDACK WINE', ['Adirondack Wine', 'ADIRONDACK WINE']),
 ('ALMA MAHLER', ['Alma Mahler', 'ALMA MAHLER']),
 ('Antonio Guterres', ['Antonio Guterres', 'Antnio Guterres'])]

In [ ]:
data_path="corpus/"

In [ ]:
f_name = 'persons_raw2norm' + ver_string
json_dp = json.dumps(persons_raw2norm)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

f_name = 'persons_norm2raw' + ver_string
json_dp = json.dumps(persons_norm2raw)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

## Company

In [44]:
df_company = get_df_match(company, 'Company', wiki_match=False, sorting_1st_letters=True)

# of candidate pairs: 3774


In [45]:
weights_company = [4,2,1,1,1]
add_match_score(df_company, weights=weights_company)

In [46]:
df_company = cut_off(df_company)

In [47]:
df_company.shape

(210, 8)

In [48]:
df_company.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Senate Judiciary Committee,Senate Judiciary committee,1.0,True,0.997527,1.0,1.0,0.999725
1,Cdc,cdc,1.0,True,0.997527,1.0,1.0,0.999725
2,Justice Department,"Justice, Department",1.0,True,0.997527,1.0,1.0,0.999725
3,Department of General Medicine,Department of Medicine,1.0,True,0.997527,1.0,1.0,0.999725
4,African-American,African American,1.0,True,0.997527,1.0,1.0,0.999725
5,Red Cross,RED CROSS,1.0,True,0.997527,1.0,1.0,0.999725
6,M.-G,M. G,1.0,True,0.997527,1.0,1.0,0.999725
7,Grubhub,GrubHub,1.0,True,0.997527,1.0,1.0,0.999725
8,The New York Times,the New York Times,1.0,True,0.997527,1.0,1.0,0.999725
9,UniCredit,Unicredit,1.0,True,0.997527,1.0,1.0,0.999725


In [49]:
df_company.tail(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
200,Huawei's,Huawei,0.933443,False,1.233946e-04,0.5,0.966667,0.577840
201,Disney,Disney's,0.932658,False,1.233946e-04,0.5,0.966667,0.577491
202,ESPN's,ESPN,0.938830,False,1.233946e-04,0.5,0.933333,0.576531
203,NASCAR,NASCAR's,0.927331,False,1.233946e-04,0.5,0.966667,0.575124
204,Atlantic,Atlantic's,0.917149,False,1.233946e-04,0.5,0.986667,0.572821
205,Rotary,Rotary's,0.918949,False,1.233946e-04,0.5,0.966667,0.571399
206,Engie,ENGIE's,0.919571,False,8.315280e-07,0.5,0.952381,0.570074
207,NCAA,NCAA's,0.924051,False,1.233946e-04,0.5,0.933333,0.569962
208,UCLA,UCLA's,0.920832,False,1.233946e-04,0.5,0.933333,0.568532
209,Boeing,Boeing's,0.908895,False,1.233946e-04,0.5,0.966667,0.566930


In [50]:
company_connected = get_connected_comp(df_company)
connected_comp = company_connected

In [51]:
connected_comp_short, connected_comp_long, connected_comp_all = get_postProcessigUnits(connected_comp)

In [52]:
print(len(connected_comp_short))
print(len(connected_comp_long))
print(len(connected_comp_all))

183
11
194


In [53]:
connected_comp_short[:20]

[(0, ['AFSCME Local 391', 'AFSCME Local 387']),
 (1, ['Ace Speedway', 'ACE Speedway']),
 (2, ['African American', 'African-American']),
 (3, ['African Development Bank', 'Asian Development Bank']),
 (4, ['Air Force', "Air Force's"]),
 (5, ["Air France-KLM's", 'Air France-KLM']),
 (6, ["Airfinance Journal's", 'Airfinance Journal']),
 (7, ['Aldi', 'ALDI']),
 (8, ['Alligator Farm', "Alligator Farm's"]),
 (9, ['AmedisysAmedisys, Inc', 'Amedisys Inc']),
 (10, ['American Airlines', "American Airlines'"]),
 (11, ['American Civil Liberties Union', "American Civil Liberties Union's"]),
 (12, ['American Legion', 'Genoa American Legion']),
 (13, ["Atlantic's", 'Atlantic']),
 (14, ['Ball State University', "Ball State University's"]),
 (15, ["Ballad Health's", 'Ballad Health']),
 (16, ['Ballou High School', "Ballou High School's"]),
 (17, ['Bank of America Merril Lynch', 'Bank of America Merrill Lynch']),
 (18, ['Baylor University', 'Taylor University']),
 (19, ["Boeing's", 'Boeing'])]

In [54]:
len(connected_comp_long)

11

In [55]:
connected_comp_long

[(0, ['Carnival Corporation', 'Carnival Corp.', 'Carnival Corp']),
 (1,
  ['Centers for Disease Control and Prevention',
   "Center for Disease Control and Prevention's",
   "Centers for Disease Control and Prevention's"]),
 (2,
  ['Department of Defence',
   'Department of Defense',
   'Department of Treasury']),
 (3,
  ["Department of Social Service's",
   'Department of Social Services',
   'Dept. of Social Services']),
 (4, ['GRUBHUB', 'Grubhub', 'GrubHub']),
 (5, ['M G', 'M. G', 'M.-G']),
 (6, ['Republicans', 'Republican Party', "Republicans'"]),
 (7, ['Un', 'UN', 'un']),
 (8, ['WHO', 'who', 'Who']),
 (9,
  ['World Health Organisation',
   'World Health  Organization',
   'World Health Organization',
   "World Health Organization's"]),
 (10, ['cdc', 'Cdc', 'CDC'])]

In [56]:
# manual insepction to excude, connect & add
index2exclude = [2]
index2connect = [(8,9)]
comp2add = ['Department of Defence',
   'Department of Defense']

In [57]:
connected_comp_final = get_comp_final(connected_comp_long, connected_comp_short, is_all=False
                                     , index2connect=index2connect
                                     , index2exclude=index2exclude
                                     , comp2add=comp2add
                                     )

In [58]:
print(len(connected_comp))
print(len(connected_comp_final))

194
194


In [59]:
company_connected = connected_comp_final

In [60]:
normalized_terms = choose_normalized(company_connected, choice='long')

In [61]:
raw2normalized, normalized2raw = make_norm_dict(company_connected, normalized_terms)

In [62]:
print("# of raw (in-snippet) entiteis normalized: {}".format(len(raw2normalized)))
print("# of normalized keys: {}".format(len(normalized2raw)))

# of raw (in-snippet) entiteis normalized: 410
# of normalized keys: 193


In [ ]:
ver_string="2"
f_name = 'company_matching' + ver_string
df_company.to_csv("corpus/" + f_name + ".csv")

In [65]:
company_raw2norm = raw2normalized
company_norm2raw = normalized2raw

In [66]:
list(company_raw2norm.items())[:5]

[('Carnival Corporation', 'Carnival Corporation'),
 ('Carnival Corp.', 'Carnival Corporation'),
 ('Carnival Corp', 'Carnival Corporation'),
 ('Centers for Disease Control and Prevention',
  "Centers for Disease Control and Prevention's"),
 ("Center for Disease Control and Prevention's",
  "Centers for Disease Control and Prevention's")]

In [67]:
list(company_norm2raw.items())[:5]

[('Carnival Corporation',
  ['Carnival Corporation', 'Carnival Corp.', 'Carnival Corp']),
 ("Centers for Disease Control and Prevention's",
  ['Centers for Disease Control and Prevention',
   "Center for Disease Control and Prevention's",
   "Centers for Disease Control and Prevention's"]),
 ("Department of Social Service's",
  ["Department of Social Service's",
   'Department of Social Services',
   'Dept. of Social Services']),
 ('GrubHub', ['GRUBHUB', 'Grubhub', 'GrubHub']),
 ('M. G', ['M G', 'M. G', 'M.-G'])]

In [ ]:
data_path="corpus/"

f_name = 'company_raw2norm' + ver_string
json_dp = json.dumps(company_raw2norm)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

f_name = 'company_norm2raw' + ver_string
json_dp = json.dumps(company_norm2raw)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

## Location

In [68]:
df_location = get_df_match(location, 'Location', wiki_match=False, sorting_1st_letters=True)

# of candidate pairs: 4622


In [69]:
weights_location = [4,2,1,1,1]
add_match_score(df_location, weights=weights_location)

In [70]:
df_location = cut_off(df_location)

In [71]:
df_location.shape

(379, 8)

In [72]:
location_connected = get_connected_comp(df_location)
connected_comp = location_connected

In [73]:
df_company.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Senate Judiciary Committee,Senate Judiciary committee,1.0,True,0.997527,1.0,1.0,0.999725
1,Cdc,cdc,1.0,True,0.997527,1.0,1.0,0.999725
2,Justice Department,"Justice, Department",1.0,True,0.997527,1.0,1.0,0.999725
3,Department of General Medicine,Department of Medicine,1.0,True,0.997527,1.0,1.0,0.999725
4,African-American,African American,1.0,True,0.997527,1.0,1.0,0.999725
5,Red Cross,RED CROSS,1.0,True,0.997527,1.0,1.0,0.999725
6,M.-G,M. G,1.0,True,0.997527,1.0,1.0,0.999725
7,Grubhub,GrubHub,1.0,True,0.997527,1.0,1.0,0.999725
8,The New York Times,the New York Times,1.0,True,0.997527,1.0,1.0,0.999725
9,UniCredit,Unicredit,1.0,True,0.997527,1.0,1.0,0.999725


In [74]:
df_company.head(10)

,in_text,matched,similarity,first_letter_match,dmtph_match,inv_lvst_dist,jw_sim,match_score
0,Senate Judiciary Committee,Senate Judiciary committee,1.0,True,0.997527,1.0,1.0,0.999725
1,Cdc,cdc,1.0,True,0.997527,1.0,1.0,0.999725
2,Justice Department,"Justice, Department",1.0,True,0.997527,1.0,1.0,0.999725
3,Department of General Medicine,Department of Medicine,1.0,True,0.997527,1.0,1.0,0.999725
4,African-American,African American,1.0,True,0.997527,1.0,1.0,0.999725
5,Red Cross,RED CROSS,1.0,True,0.997527,1.0,1.0,0.999725
6,M.-G,M. G,1.0,True,0.997527,1.0,1.0,0.999725
7,Grubhub,GrubHub,1.0,True,0.997527,1.0,1.0,0.999725
8,The New York Times,the New York Times,1.0,True,0.997527,1.0,1.0,0.999725
9,UniCredit,Unicredit,1.0,True,0.997527,1.0,1.0,0.999725


In [75]:
connected_comp_short, connected_comp_long, connected_comp_all = get_postProcessigUnits(connected_comp)

In [76]:
print(len(connected_comp_short))
print(len(connected_comp_long))
print(len(connected_comp_all))

333
19
352


In [77]:
connected_comp_short[:20]

[(0, ['12th Street', '12th St.']),
 (1, ['13th Street', '13th St.']),
 (2, ['42nd District', '2nd District']),
 (3, ['4314 39th Avenue', '4314 39th Ave.']),
 (4, ['5th Avenue', '75th Avenue']),
 (5, ['ALABAMA', 'Alabama']),
 (6, ['ALASKA', 'Alaska']),
 (7, ['ARMENIA', 'Armenia']),
 (8, ['Albany', 'ALBANY']),
 (9, ['American Legion Post 10', 'American Legion Post 1']),
 (10, ['American Legion Post 366', 'American Legion Post 333']),
 (11, ['American Legion Post 552', 'American Legion Post 502']),
 (12, ['Antwerp', 'Antwerpen']),
 (13, ['Arizona', 'ARIZONA']),
 (14, ['Arlington', 'ARLINGTON']),
 (15, ['Asia', 'ASIA']),
 (16, ['Asia Pacific', 'Asia-Pacific']),
 (17, ['Atlanta', 'ATLANTA']),
 (18, ['Austin', 'AUSTIN']),
 (19, ['Austin, TX.', 'Austin TX.'])]

In [78]:
len(connected_comp_long)

19

In [80]:
connected_comp_all

[(0, ['12th Ave. N', '12th Ave. North', '12th Avenue North']),
 (1, ['12th Street', '12th St.']),
 (2, ['13th Street', '13th St.']),
 (3, ['42nd District', '2nd District']),
 (4, ['4314 39th Avenue', '4314 39th Ave.']),
 (5, ['5th Avenue', '75th Avenue']),
 (6, ['ALABAMA', 'Alabama']),
 (7, ['ALASKA', 'Alaska']),
 (8, ['ARMENIA', 'Armenia']),
 (9, ['Albany', 'ALBANY']),
 (10, ['American Legion Post 10', 'American Legion Post 1']),
 (11, ['American Legion Post 366', 'American Legion Post 333']),
 (12, ['American Legion Post 552', 'American Legion Post 502']),
 (13, ['Antwerp', 'Antwerpen']),
 (14, ['Arizona', 'ARIZONA']),
 (15, ['Arlington', 'ARLINGTON']),
 (16, ['Asia', 'ASIA']),
 (17, ['Asia Pacific', 'Asia-Pacific']),
 (18, ['Atlanta', 'ATLANTA']),
 (19, ['Austin', 'AUSTIN']),
 (20, ['Austin, TX.', 'Austin TX.']),
 (21, ['BERKELEY', 'Berkeley']),
 (22, ['BOONE, N.C.', 'Boone, N.C.']),
 (23, ['BRAZIL', 'Brazil']),
 (24, ['Bangkok', 'BANGKOK']),
 (25, ['Bangladesh', 'BANGLADESH']),
 (2

In [81]:
# manual insepction to excude, connect & add
index2exclude = [3,5,79,82,91,299,333]
index2connect = [(10,11,12),(140,141,142),(313,314),(317,319,323,351),(336,337)]
comp2add = ['FULTON','Fulton']

In [82]:
connected_comp_final = get_comp_final(connected_comp_all, connected_comp_short, is_all=True
                                     , index2connect=index2connect
                                     , index2exclude=index2exclude
                                     , comp2add=comp2add
                                     )

In [83]:
print(len(connected_comp))
print(len(connected_comp_final))

352
338


In [84]:
location_connected = connected_comp_final

In [85]:
normalized_terms = choose_normalized(location_connected, choice='long')

In [86]:
raw2normalized, normalized2raw = make_norm_dict(location_connected, normalized_terms)

In [87]:
print("# of raw (in-snippet) entiteis normalized: {}".format(len(raw2normalized)))
print("# of normalized keys: {}".format(len(normalized2raw)))

# of raw (in-snippet) entiteis normalized: 721
# of normalized keys: 338


In [ ]:
ver_string="2"
f_name = 'location_matching' + ver_string
df_location.to_csv("corpus/" + f_name + ".csv")

In [88]:
location_raw2norm = raw2normalized
location_norm2raw = normalized2raw

In [89]:
list(location_raw2norm.items())[:5]

[('12th Ave. N', '12th Avenue North'),
 ('12th Ave. North', '12th Avenue North'),
 ('12th Avenue North', '12th Avenue North'),
 ('12th Street', '12th Street'),
 ('12th St.', '12th Street')]

In [90]:
list(location_norm2raw.items())[:5]

[('12th Avenue North',
  ['12th Ave. N', '12th Ave. North', '12th Avenue North']),
 ('12th Street', ['12th Street', '12th St.']),
 ('13th Street', ['13th Street', '13th St.']),
 ('4314 39th Avenue', ['4314 39th Avenue', '4314 39th Ave.']),
 ('ALABAMA', ['ALABAMA', 'Alabama'])]

In [ ]:
data_path="corpus/"

f_name = 'location_raw2norm' + ver_string
json_dp = json.dumps(location_raw2norm)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)

f_name = 'location_norm2raw' + ver_string
json_dp = json.dumps(location_norm2raw)
with open(data_path + f_name + ".json","w") as f:
    f.write(json_dp)